# Technology

gdsfactory inclues all the defaults for the generic Technology in `TECH` dataclass that you can customize.


## Layers

A GDS has different layers to describe the different fabrication process steps.

GDS layers have 2 integer numbers: GDSlayer, GDSpurpose

Klayout shows Layers with a color, style and transparency when showing GDS layouts.

In [ ]:
import omegaconf
import dataclasses
import pp
from pp.layers import preview_layerset

In [ ]:
layer_wg = pp.LAYER.WG
print(layer_wg)

In [ ]:
c = preview_layerset()
c

### Remove layers

You can remove layers using the `remove_layers()` function.

In [ ]:
c.remove_layers(layers=(pp.LAYER.WG, pp.LAYER.WGN))

### Remap layers

You can remap layers using the `remap_layer` to remap some layers

In [ ]:
c.remap_layers(layermap={(2,0): pp.LAYER.WGN})

### Extract layers

You can also extract layers using the `extract` function. This function returns a flattened component that contains the extracted layers.

In [ ]:
c.extract(layers=(pp.LAYER.WGN,))

## TECH

As you can see in `pp.tech.TECH` you can define all your technology constants in dataclasses. 
This is a convenient way to group all the specific constants for a technology. 

In [ ]:
import pp

waveguide_pn = pp.tech.Waveguide(
    width=0.5,
    layer=pp.LAYER.WG,
    sections=(
        pp.tech.Section(width=2, layer=pp.LAYER.N, offset=+1),
        pp.tech.Section(width=2, layer=pp.LAYER.P, offset=-1),
    ))

pp.TECH.waveguide.pn = waveguide_pn

In [ ]:
c = pp.components.straight(waveguide='pn')
c

## Library

How can you customize components for a particular Fab technology?

You can customize the `TECH` values for your particular foundry process

`PDK` stands for Process Design Kit, where you can define:

- Process (Layers for each fabrication step, Design Rules, Waveguide CrossSection ...) in `TECH` `pp.tech.Tech`
- Library of components optimized for that particular process. You can create a `Library` for that particular technology


A library

In [ ]:
import pp

c = pp.components.straight(waveguide='nitride')
c

In [ ]:
wg = pp.components.straight(waveguide='nitride')
gc = pp.components.grating_coupler_elliptical_te(layer=pp.TECH.waveguide.nitride.layer, wg_width=pp.TECH.waveguide.nitride.width)
wg_gc = pp.routing.add_fiber_single(component=wg, grating_coupler=gc, waveguide='nitride')
wg_gc

### FabA

Lets add to TECH the specific waveguide needed for FabA

FabA only has one Metal layer available that is defined in GDS layer (30, 0)

The metal layer traces are 2um wide

In [ ]:
import dataclasses
import pp
from pp.tech import TECH, Layer, Waveguide


@dataclasses.dataclass
class Metal1(Waveguide):
    width: float = 2.0
    width_wide: float = 10.0
    auto_widen: bool = False
    layer: Layer = (30, 0)
    radius: float = 10.0


METAL1 = Metal1()
TECH.waveguide.metal1 = METAL1


wg = pp.components.straight(length=20, waveguide="metal1")
gc = pp.components.grating_coupler_elliptical_te(
    layer=METAL1.layer, wg_width=METAL1.width
)

wg_gc = pp.routing.add_fiber_array(
    component=wg, grating_coupler=gc, waveguide="metal1"
)
wg_gc

In [ ]:
coupler = pp.components.coupler(waveguide='metal1')
coupler

In [ ]:
from functools import partial

In [ ]:
coupler = partial

### FabB

FabB has photonic waveguides that require many cladding layers to avoid dopants

Lets say that the waveguides are defined in layer (2, 0) and are 0.3um wide

In [ ]:
from typing import Tuple
import dataclasses
from pp.tech import LAYER, TECH, Layer, Waveguide
import pp


@dataclasses.dataclass
class StripB(Waveguide):
    width: float = 0.3
    width_wide: float = 10.0
    auto_widen: bool = False
    layer: Layer = (2, 0)
    radius: float = 10.0
    layers_cladding: Tuple[Layer] = ((71, 0), (68, 0))


STRIPB = StripB()
TECH.waveguide.stripb = STRIPB
wg = pp.components.straight(length=20, waveguide="stripb")
gc = pp.components.grating_coupler_elliptical_te(layer=STRIPB.layer, wg_width=STRIPB.width)
wg_gc = pp.routing.add_fiber_array(
    component=wg, grating_coupler=gc, waveguide="stripb"
)
wg_gc

### FabC

Lets assume that fab C has both Silicon and Silicon Nitride components, and you need different waveguide widths for C and O band.

Lets asume that O band nitride waveguide width is 0.9 and Cband Nitride waveguide width is 1um, and for 0.4um for Silicon O band and 0.5um for silicon Cband.

Lets also that this foundry has an LVS flow where all components have optical pins defined in layer (100, 0)

In [ ]:
from typing import Callable, Dict, Optional, Tuple

import pydantic
import pydantic.dataclasses as dataclasses

import pp
from pp.add_pins import add_pin_square_inside
from pp.component import Component, ComponentReference
from pp.tech import TECH, LayerLevel, LayerStack, Library, Tech, Waveguide
from pp.types import Layer


@dataclasses.dataclass
class LayerMap:
    WG: Layer = (10, 1)
    WG_CLAD: Layer = (10, 2)
    WGN: Layer = (34, 0)
    WGN_CLAD: Layer = (36, 0)
    PIN: Layer = (100, 0)


LAYER = LayerMap()
WIDTH_NITRIDE_OBAND = 0.9
WIDTH_NITRIDE_CBAND = 1.0
PORT_TYPE_TO_LAYER = dict(optical=(100, 0))


@pydantic.dataclasses.dataclass
class LayerStackFabc(LayerStack):
    WGN = LayerLevel(
        layer=(34, 0), thickness_nm=350.0, zmin_nm=220.0 + 100.0, material="sin"
    )
    WGN_CLAD = LayerLevel(layer=(36, 0))


@dataclasses.dataclass
class StripNitrideCband(Waveguide):
    width: float = WIDTH_NITRIDE_CBAND
    layer: Layer = LAYER.WGN
    auto_widen: bool = False
    radius: float = 10.0
    layers_cladding: Tuple[Layer, ...] = (LAYER.WGN_CLAD,)


@dataclasses.dataclass
class StripNitrideOband(StripNitrideCband):
    width: float = WIDTH_NITRIDE_OBAND


NITRIDE_CBAND = StripNitrideCband()
NITRIDE_OBAND = StripNitrideOband()

TECH.waveguide.fabc_nitride_cband = NITRIDE_CBAND
TECH.waveguide.fabc_nitride_oband = NITRIDE_OBAND


def add_pins(
    component: Component,
    reference: Optional[ComponentReference] = None,
    function: Callable = add_pin_square_inside,
    port_type_to_layer: Dict[str, Tuple[int, int]] = PORT_TYPE_TO_LAYER,
    pin_length: float = 0.5,
    **kwargs,
) -> None:
    """Add Pin port markers.

    Args:
        component: to add ports
        function:
        port_type_to_layer: dict mapping port types to marker layers for ports

    """
    reference = reference or component
    for p in reference.ports.values():
        if p.port_type in port_type_to_layer:
            layer = port_type_to_layer[p.port_type]
            function(
                component=component,
                port=p,
                layer=layer,
                label_layer=layer,
                pin_length=pin_length,
                **kwargs,
            )


mmi1x2_nitride_c = pp.partial(
    pp.c.mmi1x2,
    width=WIDTH_NITRIDE_CBAND,
    waveguide="fabc_nitride_cband",
    post_init=add_pins,
)
mmi1x2_nitride_o = pp.partial(
    pp.c.mmi1x2,
    width=WIDTH_NITRIDE_OBAND,
    waveguide="fabc_nitride_oband",
    post_init=add_pins,
)
bend_euler_c = pp.partial(
    pp.c.bend_euler, waveguide="fabc_nitride_cband", post_init=add_pins
)
straight_c = pp.partial(
    pp.c.straight, waveguide="fabc_nitride_cband", post_init=add_pins
)
bend_euler_o = pp.partial(
    pp.c.bend_euler, waveguide="fabc_nitride_oband", post_init=add_pins
)
straight_o = pp.partial(
    pp.c.straight, waveguide="fabc_nitride_oband", post_init=add_pins
)

mzi_nitride_c = pp.partial(
    pp.c.mzi,
    waveguide="fabc_nitride_cband",
    splitter=mmi1x2_nitride_c,
    post_init=add_pins,
    straight=straight_c,
    bend=bend_euler_c,
)
mzi_nitride_o = pp.partial(
    pp.c.mzi,
    waveguide="fabc_nitride_oband",
    splitter=mmi1x2_nitride_c,
    post_init=add_pins,
    straight=straight_o,
    bend=bend_euler_o,
)


gc_nitride_c = pp.partial(
    pp.components.grating_coupler_elliptical_te,
    grating_line_width=0.6,
    wg_width=WIDTH_NITRIDE_CBAND,
    layer=LAYER.WGN,
    post_init=add_pins,
)


TECH_FABC = Tech(name="fab_c")
LIBRARY = Library(name="fab_c")
LIBRARY.register(
    [
        mmi1x2_nitride_c,
        mmi1x2_nitride_o,
        bend_euler_c,
        straight_c,
        mzi_nitride_c,
        mzi_nitride_o,
    ]
)

In [ ]:
mzi = mzi_nitride_c()
mzi_gc = pp.routing.add_fiber_single(
    component=mzi,
    grating_coupler=gc_nitride_c,
    waveguide="fabc_nitride_cband",
    optical_routing_type=1,
    straight_factory=straight_c,
    bend_factory=bend_euler_c,
)
mzi_gc